# Parse JSON

In [63]:
import json
import re

In [2]:
with open('menu.json','r') as f:
    menu = json.load(f)

In [14]:
location = menu.pop('Location')
menus = menu.pop('Menus')

In [17]:
index2el = {0:'name',1:'price',2:'moreInfo'}

In [18]:
for k,v in menu.items():
    for food,info in v.items():
        v[food] = {index2el[i]:el for i,el in enumerate(info)}

In [20]:
!pip install flatten_json

  Obtaining dependency information for flatten_json from https://files.pythonhosted.org/packages/63/b5/99f20a19b839e04fffab924be192681b797b40bcf83abdfa508371c6273c/flatten_json-0.1.14-py3-none-any.whl.metadata


In [21]:
from flatten_json import flatten

In [22]:
menu_flatten = flatten(menu)

In [23]:
menu_flatten

{'Chicken_C1_name': 'Original Recipe',
 'Chicken_C1_price': 3.5,
 'Chicken_C1_moreInfo_nutritionalInfo_kcal': 400,
 'Chicken_C1_moreInfo_nutritionalInfo_fat': 22,
 'Chicken_C1_moreInfo_nutritionalInfo_protein': 28,
 'Chicken_C1_moreInfo_nutritionalInfo_itemId': 4,
 'Chicken_C1_moreInfo_nutritionalInfo_allergens_0': 'wheat',
 'Chicken_C1_moreInfo_nutritionalInfo_allergens_1': 'soy',
 'Chicken_C1_moreInfo_available': False,
 'Chicken_C2_name': 'Popcorn Chicken',
 'Chicken_C2_price': 4,
 'Chicken_C2_moreInfo_nutritionalInfo_kcal': 350,
 'Chicken_C2_moreInfo_nutritionalInfo_fat': 20,
 'Chicken_C2_moreInfo_nutritionalInfo_protein': 25,
 'Chicken_C2_moreInfo_nutritionalInfo_itemId': 6,
 'Chicken_C2_moreInfo_nutritionalInfo_allergens_0': 'wheat',
 'Chicken_C2_moreInfo_nutritionalInfo_allergens_1': 'soy',
 'Chicken_C2_moreInfo_available': False,
 'Chicken_C4_name': 'Hot Wings',
 'Chicken_C4_price': 3,
 'Chicken_C4_moreInfo_nutritionalInfo_kcal': 270,
 'Chicken_C4_moreInfo_nutritionalInfo_fat':

In [27]:
for m,info in menus.items():
    for i,con in enumerate(info['contents']):
        if isinstance(con,list):
            info['contents'][i] = {'item':con[0],'amount':con[1]}
        

In [29]:
menus_flatten = flatten(menus)

In [33]:
flattened_full_menu = {**menu_flatten,**menus_flatten}
len(flattened_full_menu)

686

In [36]:
def break_dict_into_chunks(dictionary, chunk_size):
    chunks = []
    current_chunk = {}
    count = 0

    for key, value in dictionary.items():
        current_chunk[key] = value
        count += 1
        if count == chunk_size:
            chunks.append(current_chunk)
            current_chunk = {}
            count = 0

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [51]:
context_list = break_dict_into_chunks(flattened_full_menu,10)

In [76]:
def get_context_list(menu,chunks,win):
    context_list = break_dict_into_chunks(menu,chunks)
    
    for i in range(1,len(context_list)):
        prev = context_list[i-1]
        window = {k:v for k,v in list(prev.items())[-win:]}
        context_list[i] = {**window,**context_list[i]}
    
    for i,dic in enumerate(context_list):
        con = 'passage: '
        for k,v in dic.items():
            con += re.sub(r'_',': ',k)+': '+str(v)+', '
        context_list[i] = con
    
    return context_list

In [77]:
context_list = get_context_list(flattened_full_menu,10,2)

In [78]:
context_list

['passage: Chicken: C1: name: Original Recipe, Chicken: C1: price: 3.5, Chicken: C1: moreInfo: nutritionalInfo: kcal: 400, Chicken: C1: moreInfo: nutritionalInfo: fat: 22, Chicken: C1: moreInfo: nutritionalInfo: protein: 28, Chicken: C1: moreInfo: nutritionalInfo: itemId: 4, Chicken: C1: moreInfo: nutritionalInfo: allergens: 0: wheat, Chicken: C1: moreInfo: nutritionalInfo: allergens: 1: soy, Chicken: C1: moreInfo: available: False, Chicken: C2: name: Popcorn Chicken, ',
 'passage: Chicken: C1: moreInfo: available: False, Chicken: C2: name: Popcorn Chicken, Chicken: C2: price: 4, Chicken: C2: moreInfo: nutritionalInfo: kcal: 350, Chicken: C2: moreInfo: nutritionalInfo: fat: 20, Chicken: C2: moreInfo: nutritionalInfo: protein: 25, Chicken: C2: moreInfo: nutritionalInfo: itemId: 6, Chicken: C2: moreInfo: nutritionalInfo: allergens: 0: wheat, Chicken: C2: moreInfo: nutritionalInfo: allergens: 1: soy, Chicken: C2: moreInfo: available: False, Chicken: C4: name: Hot Wings, Chicken: C4: price

# Make embeddings

In [80]:
!pip install sentence_transformers

  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/b5/d0/ba1577e198681c76125a810217b305a7010a9280c24c314c7dd7d09d62ab/sentence_transformers-2.5.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 1.6 MB/s eta 0:00:0000:010:01


In [81]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('intfloat/multilingual-e5-large')
embeddings = model.encode(context_list, normalize_embeddings=True)


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [83]:
!pip install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/cc/63/b7d76109331318423f9cfb89bd89c99e19f5d0b47a5105439a629224d297/chromadb-0.4.24-py3-none-any.whl.metadata
  Obtaining dependency information for build>=1.0.3 from https://files.pythonhosted.org/packages/06/d5/5ff223d89a6c461565ad06f5fdc089dcf7cc88283b9d8b84a11a80526927/build-1.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/94/3f/844393b0d2ea1072b7704d6eff5c595e05ae8b831b96340cdb76b2fe995c/chroma_hnswlib-0.7.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/f0/f7/ea860cb8aa18e326f411e32ab537424690a53db20de6bad73d70da611fae/fastapi-0.110.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/d9/fd/bac111726b6c651f1fa5563145ecba5ff

  Obtaining dependency information for pyproject_hooks from https://files.pythonhosted.org/packages/d5/ea/9ae603de7fbb3df820b23a70f6aff92bf8c7770043254ad8d2dc9d6bcba4/pyproject_hooks-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for starlette<0.37.0,>=0.36.3 from https://files.pythonhosted.org/packages/eb/f7/372e3953b6e6fbfe0b70a1bb52612eae16e943f4288516480860fcd4ac41/starlette-0.36.3-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.5.0 from https://files.pythonhosted.org/packages/f9/de/dc04a3ea60b22624b51c703a84bbe0184abcd1d0b9bc8074b5d6b7ab90bb/typing_extensions-4.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth>=1.0.1 from https://files.pythonhosted.org/packages/b7/1d/f152a5f6d243b6acbb2a710ed19aa47154d678359bed995abdd9daf0cff0/google_auth-2.28.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for requests-oauthlib from https://files.pythonhosted.org/packages/6f/bb/5deac77a9af

  Obtaining dependency information for asgiref~=3.0 from https://files.pythonhosted.org/packages/9b/80/b9051a4a07ad231558fcd8ffc89232711b4e618c15cb7a392a17384bbeef/asgiref-3.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for monotonic>=1.5 from https://files.pythonhosted.org/packages/9a/67/7e8406a29b6c45be7af7740456f7f37025f0506ae2e05fb9009a53946860/monotonic-1.6-py2.py3-none-any.whl.metadata
  Obtaining dependency information for httptools>=0.5.0 from https://files.pythonhosted.org/packages/80/dd/cebc9d4b1d4b70e9f3d40d1db0829a28d57ca139d0b04197713816a11996/httptools-0.6.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for python-dotenv>=0.13 from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for uvloop!=0.15.0,!=0.15.1,>=0.14.0 from https://files.pythonhosted.org/packages/13/00/d0923d66d80c871798

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 15.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 21.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 17.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.9/186.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

# Add embeddings to Vector DB

In [85]:
import chromadb
client = chromadb.PersistentClient('.')

In [99]:
 collection = client.create_collection(
        name="Restaurant_DB",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [100]:
collection.add(
    embeddings=embeddings.tolist(),
    documents=context_list,
    ids=['id'+str(i) for i in range(len(embeddings))]
)

# Inference 

In [136]:
def get_answer(query,n,collection,model_emb,model_llm,openAI):
    emb_query = model_emb.encode('query: '+query,normalize_embeddings=True)
    res = collection.query(
        query_embeddings=emb_query.tolist(),
        n_results = n
    ) 
    context = ''.join(con[9:] for con in res['documents'][0])
    
    prompt = f'''You are a waiter for a restaurant. 
    You will be provided with a question and context. 
    The context will contain information about the menu. 
    Answer the question only using the context provided; if the context does not contain the infromation please answer that you dont know the answer to the question.
    
    Question: {query}
    Context: {context}'''
    #print(prompt)
    
    if model_llm:
        response = model_llm(prompt,return_full_text=False)
        return response,context
    elif openAI:
        res = openAI.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
          ]
        )

        content = res.choices[0].message.content
        return content,context
    else: 
        print('no llm')
        return prompt

    

In [ ]:
import chromadb
client = chromadb.PersistentClient('.')

In [ ]:
collection = client.get_collection('Restaurant_DB')

In [164]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
model_emb = SentenceTransformer('intfloat/multilingual-e5-large')
#pipe_llm = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")

In [127]:
'''from openai import OpenAI

client = OpenAI(api_key='.....')'''

In [165]:
query = 'do you still have the popcorn chicken?'

In [166]:
con = get_answer(query,3,collection,model_emb,model_llm=None,openAI=None)

no llm


In [167]:
con

'You are a waiter for a restaurant. \n    You will be provided with a question and context. \n    The context will contain information about the menu. \n    Answer the question only using the context provided; if the context does not contain the infromation please answer that you dont know the answer to the question.\n    \n    Question: do you still have the popcorn chicken?\n    Context: Chicken: C1: name: Original Recipe, Chicken: C1: price: 3.5, Chicken: C1: moreInfo: nutritionalInfo: kcal: 400, Chicken: C1: moreInfo: nutritionalInfo: fat: 22, Chicken: C1: moreInfo: nutritionalInfo: protein: 28, Chicken: C1: moreInfo: nutritionalInfo: itemId: 4, Chicken: C1: moreInfo: nutritionalInfo: allergens: 0: wheat, Chicken: C1: moreInfo: nutritionalInfo: allergens: 1: soy, Chicken: C1: moreInfo: available: False, Chicken: C2: name: Popcorn Chicken, Chicken: C1: moreInfo: available: False, Chicken: C2: name: Popcorn Chicken, Chicken: C2: price: 4, Chicken: C2: moreInfo: nutritionalInfo: kcal:

## Discussion

the function get_answer() is the only function needed, other wise the Inference chapter covers how to use the RAG. Sadly i couldnt use Mistral due to my pc not having enough ram, however i used gpt 3.5 a bit to test it and get the answers below.

- do you have cola? answer: I am sorry, but based on the information provided in the context, there is no mention of cola on the drinks menu.
- How many calories does the Colonel have? answer: The Colonel Burger has 150 calories.
- Can I get a Whopper? answer: Sorry, the Whopper is not available on the menu.
- do you still have the popcorn chicken? answer: I'm sorry, but the Popcorn Chicken is not available.

performs pretty well when asked about factual information about the food. On the other side it performs not that well when given an assertion for example: Give me a Veggie Tender, medium, with salad. I believe this is because the context returned is for the veggie tender however this context also can contain referecne to other products like 'any side dish' and the context for the side dishes is not returned so it cant know that salad is a side dish. I think an improvement can be made here where there can be a mapping between these.

the vector db stores the embeddings for the contexts (the json file parsed), however when one adds a query the embedding for that query has to be made which can take a few seconds; otherwise the cosine similarity between the embedding of the query and those of the contexts and the generation of the prompt is almost instant.

With a bit more time i wouldve liked to have more of a thought of what better way to parse the json to extract context that makes even more sense and can produce even more meaningful embeddings.